In [25]:
import pandas as pd
import json

# チームIDをチーム名に変換する辞書作成
team_df = pd.read_csv('/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/team.csv')
team_dic = dict(zip(team_df['team_id'], team_df['team_name']))
team_id_list = [v for v in team_dic.keys()]


# ○: フル出場, ▲: 途中出場, ▽: 途中退場, ◆: 途中出場途中退場, ×: 出場停止, ※: 他大会の影響で出場停止, B: ベンチ入り
# 各々の回数をカウント
tmp = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
colnames = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
characters = ['○', '▲', '▽', '◆', 'B', '×']


#記号を出場時間に変換する関数を定義
def convert_sign(self):
    import re
    
    if "○" in self:
        return int(90)
    
    elif "▽" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)

    elif "▲" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)    
    
    elif "◆" in self:        
            num = re.sub(r"\D", "", self)
            return int(num)
    
    else:
        return int(0)

pretime_2018 = pd.DataFrame({"name":[], "pretime":[]})
pretime_2017 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2016 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2015 = pd.DataFrame({"name":[], "pretime":[]})    

df_dic = {2018:pretime_2018, 2017:pretime_2017, 2016:pretime_2016, 2015:pretime_2015}
for year in [2018,2017,2016,2015]:
    # 2015シーズンのjsonをcsvに変換
    #year = 2015
    team_df_year = team_df[team_df.year == year]
    df = pd.DataFrame({"name":[],
                     "pretime":[]})
    for div, team_id in zip(team_df_year["div"], team_df_year["team_id"]):
        # 2stageかどうか
        is_2stage = year < 2017 and div == "J1"

        # 後で１試合あたりの指標に変換するために試合数の情報を記憶
        if div == 'J1':
            num_match = 34
        else:
            num_match = 42

        if is_2stage:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s1.json", encoding="utf-8_sig") as f:
                first_dic = json.load(f)
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s2.json", encoding="utf-8_sig") as f:
                second_dic = json.load(f)

            first_df = pd.DataFrame.from_dict(first_dic, orient='index')
            second_df = pd.DataFrame.from_dict(second_dic, columns=list(range(17, 34)), orient='index')
            #(first_df)
            full_df = pd.concat([first_df, second_df], axis=1, join='outer')

        else:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}.json", encoding="utf-8_sig") as f:
                dic = json.load(f)

            full_df = pd.DataFrame.from_dict(dic, orient='index')

        #４シーズンに分けるための数値作成
        num_to_div_list = [(num_match//4)*n for n in range(1,5)]
        first = num_to_div_list[0]
        second = num_to_div_list[1]
        third = num_to_div_list[2]
        forth = num_match
        
        #関数を適用し、かつ4シーズンに分ける
        full_df = full_df.fillna("0")    
        full_df_con = full_df.applymap(convert_sign)
        full_df_con["pretime"] = full_df_con.sum(axis=1)
        full_df_con["1st_pretime"] = full_df_con.iloc[:, :first].sum(axis=1)
        full_df_con["2st_pretime"] = full_df_con.iloc[:, first:second].sum(axis=1)
        full_df_con["3st_pretime"] = full_df_con.iloc[:, second:third].sum(axis=1)
        full_df_con["4st_pretime"] = full_df_con.iloc[:, third:num_match].sum(axis=1)

        new_df = pd.DataFrame()
        new_df["name"] = full_df_con.index
        new_df["pretime"] = full_df_con["pretime"].values
        new_df["1st_pretime"] = full_df_con["1st_pretime"].values
        new_df["2st_pretime"] = full_df_con["2st_pretime"].values
        new_df["3st_pretime"] = full_df_con["3st_pretime"].values
        new_df["4st_pretime"] = full_df_con["4st_pretime"].values
        new_df['team_id'] = team_id
        new_df['team'] = new_df['team_id'].map(int).map(team_dic)
        df = pd.concat([df, new_df])

 
    df["name-team"] = df["name"] + "-" + df["team"]
    df.drop(["name"], axis=1, inplace=True)
    df.drop(["team"], axis=1, inplace=True)


    #出場時間を出場割合に変換
    df["time_rate"] = df["pretime"].copy()
    df["1st_time_rate"] = df["1st_pretime"].copy()
    df["2st_time_rate"] = df["2st_pretime"].copy()
    df["3st_time_rate"] = df["3st_pretime"].copy()
    df["4st_time_rate"] = df["4st_pretime"].copy()
    for i in team_id_list:
        s = df.loc[df["team_id"]==i, "pretime"].sum()
        df.loc[df["team_id"]==i, "time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "1st_pretime"].sum()
        df.loc[df["team_id"]==i, "1st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "2st_pretime"].sum()
        df.loc[df["team_id"]==i, "2st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "3st_pretime"].sum()
        df.loc[df["team_id"]==i, "3st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "4st_pretime"].sum()
        df.loc[df["team_id"]==i, "4st_time_rate"] /= (s/100)
        
    df.drop(["pretime"], axis=1, inplace=True)
    df.drop(["1st_pretime"], axis=1, inplace=True)
    df.drop(["2st_pretime"], axis=1, inplace=True)
    df.drop(["3st_pretime"], axis=1, inplace=True)
    df.drop(["4st_pretime"], axis=1, inplace=True)
    df.drop(["team_id"], axis=1, inplace=True)
    df_dic[year] = df
    
pretime_2018 = df_dic[2018]
pretime_2017 = df_dic[2017]
pretime_2016 = df_dic[2016]
pretime_2015 = df_dic[2015]

In [3]:
pretime_2018.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2018.csv", index=False)
pretime_2017.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2017.csv", index=False)
pretime_2016.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2016.csv", index=False)
pretime_2015.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2015.csv", index=False)


In [4]:
a = pd.read_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2018.csv")
a.head(3)

,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,クォン スンテ-鹿島,7.222041,9.090909,6.829392,4.545455,8.181818
1,内田 篤人-鹿島,1.789164,2.032828,4.122929,1.123737,0.262626
2,昌子 源-鹿島,4.095462,6.818182,5.526748,0.517677,3.636364


In [5]:
pretime_2018

,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,クォン スンテ-鹿島,7.222041,9.090909,6.829392,4.545455,8.181818
1,内田 篤人-鹿島,1.789164,2.032828,4.122929,1.123737,0.262626
2,昌子 源-鹿島,4.095462,6.818182,5.526748,0.517677,3.636364
3,レオ シルバ-鹿島,4.615568,2.310606,3.515872,7.954545,4.666667
4,植田 直通-鹿島,3.744762,9.090909,6.829392,0.000000,0.000000
...,...,...,...,...,...,...
30,永坂 勇人-水戸,0.024072,0.000000,0.101010,0.000000,0.000000
31,伊藤 涼太郎-水戸,4.111502,0.740140,3.414141,6.868687,5.194040
32,バティスタ-水戸,0.982139,0.000000,0.000000,0.333333,3.156831
33,齋藤 恵太-水戸,0.524770,0.892223,1.101010,0.141414,0.058928


In [6]:
full_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,○,○
田向 泰輝,,▽56,B,B,○,B,B,B,B,○,...,○,○,○,○,○,○,○,○,○,▽77
田中 雄大,○,○,○,○,○,○,▽89,○,○,○,...,○,○,○,○,○,○,○,○,○,○
山﨑 貴雅,,B,,,B,B,▲1,B,,,...,,,,,,,,,,
金 聖基,B,○,B,○,○,○,○,○,○,○,...,B,B,B,B,,,,,,
石神 幸征,◆71,×,○,○,B,○,○,○,▽89,▲11,...,○,○,B,▲1,○,▽75,○,,,
鈴木 雄斗,▽65,▽69,▽56,B,B,▲31,B,B,○,▽73,...,○,▽79,▽77,○,▽81,○,○,▽69,○,○
岩尾 憲,○,○,○,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,×,○
小谷野 顕治,B,B,▲25,B,B,B,B,▲8,B,▽59,...,,▲11,,,B,B,B,B,B,B
船谷 圭祐,,,,,,,,,▲9,▽79,...,▲37,▽67,○,▽72,▽78,▽89,▽76,▲34,○,▽80


In [23]:
#出場データから出場時間割合データ作成＋記号の回数をカウントのための操作確認
full_df[ "total_○"] = full_df.apply(lambda d: d.str.contains("○")).sum(axis=1).values
full_df["1st_○"] = full_df.iloc[:, :first].apply(lambda d: d.str.contains("○")).sum(axis=1).values

In [24]:
full_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,total_○,1st_○
本間 幸司,B,B,B,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,34,7
田向 泰輝,,▽56,B,B,○,B,B,B,B,○,...,○,○,○,○,○,○,○,▽77,25,2
田中 雄大,○,○,○,○,○,○,▽89,○,○,○,...,○,○,○,○,○,○,○,○,39,9


In [12]:
first

10